In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
# %matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
# pd.options.display.float_format = '{:.7g}'.format 
vTest = 1000

import torch.backends.cudnn as cudnn
cudnn.benchmark = True  
cudnn.deterministic = True

xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( device )
# if torch.cuda.is_available():
#     torch.cuda.caching_allocator_delete
#     torch.cuda.empty_cache()  # 释放显存
#     print('Memory Allocated', torch.cuda.memory_allocated() )

cuda:0


In [2]:
sql="SELECT * FROM ds_qf60"
# sql="SELECT * FROM ds_qf60_2"
df = pd.read_sql_query(sql, conn, index_col=None)
del df['index']

In [3]:
""" 确定训练的目标 """
# df['class0'] = df['test5_8']
# df['class0'] = df['test10_10']
df['class0'] = df['test20_20']
# df['class0'] = df['test20_30']
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101148 entries, 0 to 101147
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   pct_change    101148 non-null  float64
 1   diff          101148 non-null  float64
 2   dea           101148 non-null  float64
 3   bar           101148 non-null  float64
 4   jx_days_ud60  101148 non-null  int64  
 5   jx_xl_250     101148 non-null  float64
 6   jx_xl_120     101148 non-null  float64
 7   jx_xl_60      101148 non-null  float64
 8   jx_xl_20      101148 non-null  float64
 9   jx_xl_10      101148 non-null  float64
 10  jx_xl_5       101148 non-null  float64
 11  jx_zs_5       101148 non-null  int64  
 12  jx_zs_10      101148 non-null  int64  
 13  jx_zs_20      101148 non-null  int64  
 14  jx_zs_60      101148 non-null  int64  
 15  lj_fl_1_3     101148 non-null  float64
 16  lj_fl_3_10    101148 non-null  float64
 17  lj_fl_5_20    101148 non-null  float64
 18  lj_f

(None, (101148, 27))

In [4]:
df.class0.unique(), df.class0.value_counts()

(array([0., 1.]),
 0.0    91728
 1.0     9420
 Name: class0, dtype: int64)

In [5]:
#  'diff', 'dea',, 
X_columns = ['pct_change', 
             'diff','dea', 
             'jx_days_ud60', 
             'jx_xl_250', 'jx_xl_120','jx_xl_60', 
             'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 
             'jx_zs_5', 'jx_zs_10','jx_zs_20', 'jx_zs_60', 
             'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20','lj_fl_20_60']

In [6]:
X_data = df[X_columns]  
# X = torch.from_numpy(X_data.values).type(torch.float32)
Y_data = df.class0.values.reshape(-1,1)  # 转换成 pd type array
# Y = torch.from_numpy(Y_data).type(torch.float32)
# X.shape, Y.shape

In [7]:
# Y_data.class0.unique()
# df.class0.value_counts()
# Y_data


In [8]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

class Model4(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 256)   # X.size(1)= 20； 定义256个后续中间层
        self.liner_2 = nn.Linear(256, 128)  # 接着，再定义128个后续中间层
        self.liner_3 = nn.Linear(128, 32)   # 接着，再定义32个后续中间层
        self.liner_4 = nn.Linear(32, 1)     
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x
    
class Model5(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 256)   # X.size(1)= 20； 定义256个后续中间层
        self.liner_2 = nn.Linear(256, 128)  # 接着，再定义128个后续中间层
        self.liner_3 = nn.Linear(128, 128)  # 接着，再定义128个后续中间层
        self.liner_4 = nn.Linear(128, 32)   # 接着，再定义32个后续中间层
        self.liner_5 = nn.Linear(32, 1)     
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.relu( self.liner_4(x) )
        x = F.sigmoid( self.liner_5(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(29, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 64)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [9]:
# Ver 4.2 - sklearn mode

lr = 0.00012
def get_model():
    model = Model4()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

model, optim = get_model()
model = model.to(device)
# model, optim 

loss_fn = nn.BCELoss()
# loss_fn = loss_fn.to(device)
batch = 32
# batch = 64
# batch = 1024
# batch = 8192
# batch = 16384
# batch = 32768
epochs = 10
no_of_batches = len(X_data)//batch
from torch.utils.data import TensorDataset, DataLoader

def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)

train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32).to(device)
test_y = torch.from_numpy(test_y).type(torch.float32).to(device)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

train_x.shape, test_x.shape, train_y.shape, test_y.shape

(torch.Size([75861, 18]),
 torch.Size([25287, 18]),
 torch.Size([75861, 1]),
 torch.Size([25287, 1]))

In [10]:
# 

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [11]:
%%time
# epochs = 10
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:  0  train_loss:  0.3077  Acc： 0.9075 0  test_loss:  0.3121  Acc： 0.905
epoch:  1  train_loss:  0.3083  Acc： 0.9076 1  test_loss:  0.3125  Acc： 0.9048
epoch:  2  train_loss:  0.3035  Acc： 0.9075 2  test_loss:  0.3091  Acc： 0.9048
epoch:  3  train_loss:  0.3026  Acc： 0.9077 3  test_loss:  0.3084  Acc： 0.9048
epoch:  4  train_loss:  0.3021  Acc： 0.9076 4  test_loss:  0.3083  Acc： 0.905
epoch:  5  train_loss:  0.3029  Acc： 0.9075 5  test_loss:  0.3095  Acc： 0.9049
epoch:  6  train_loss:  0.3016  Acc： 0.9077 6  test_loss:  0.308  Acc： 0.9048
epoch:  7  train_loss:  0.3025  Acc： 0.9076 7  test_loss:  0.3086  Acc： 0.9049
epoch:  8  train_loss:  0.3011  Acc： 0.9076 8  test_loss:  0.3076  Acc： 0.9049
epoch:  9  train_loss:  0.3039  Acc： 0.9075 9  test_loss:  0.3111  Acc： 0.905
CPU times: total: 11.4 s
Wall time: 49.8 s


In [12]:
accuracy(model(test_x), test_y)

tensor(0.9050, device='cuda:0')

In [13]:
df.head(1)

,pct_change,diff,dea,bar,jx_days_ud60,jx_xl_250,jx_xl_120,jx_xl_60,jx_xl_20,jx_xl_10,...,lj_fl_5_20,lj_fl_20_60,test5_8,test10_10,test20_10,test20_20,test20_30,ts_code,trade_date,class0
0,-1.1538,-0.108607,-0.190289,0.163366,32,-0.000964,0.001425,-0.00262,0.000591,0.001576,...,1.44158,0.865133,0.0,0.0,0.0,0.0,0,000001.SZ,20230419,0.0


In [14]:

column_str = 'pct_change, diff, dea, jx_days_ud60, \
jx_xl_250, jx_xl_120, jx_xl_60,\
jx_xl_20, jx_xl_10, jx_xl_5, \
jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,\
lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, \
sz_max_5, sz_max_10, sz_max_20 '
# print(column_str)

sql_new= "SELECT " + column_str +" \
from day_zhibiao \
where trade_date>='20230601' and sz_max_20>20 and \
jx_days_ud60 > 20 and jx_days_ud60 < 60 and \
jx_xl_60>-"+str(xl_dgree)+" and  \
jx_xl_20>0 "
print(sql_new)

df_now = pd.read_sql_query(sql_new, conn)

df_now['class0'] = df_now['sz_max_20'] > 20 
df_now
# df_now[ df_now['class0']==1 ]
# df_now[ df['class0'] ]
# print( df_now.shape, df_now.head(1) )

SELECT pct_change, diff, dea, jx_days_ud60, jx_xl_250, jx_xl_120, jx_xl_60,jx_xl_20, jx_xl_10, jx_xl_5, jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, sz_max_5, sz_max_10, sz_max_20  from day_zhibiao where trade_date>='20230601' and sz_max_20>20 and jx_days_ud60 > 20 and jx_days_ud60 < 60 and jx_xl_60>-0.00523 and  jx_xl_20>0 


,pct_change,diff,dea,jx_days_ud60,jx_xl_250,jx_xl_120,jx_xl_60,jx_xl_20,jx_xl_10,jx_xl_5,...,jx_zs_20,jx_zs_60,lj_fl_1_3,lj_fl_3_10,lj_fl_5_20,lj_fl_20_60,sz_max_5,sz_max_10,sz_max_20,class0
0,0.8649,0.023673,-0.026637,47,0.001256,0.000402,-0.002363,0.002876,0.001426,0.004800,...,13,0,1.213370,1.336500,1.097930,0.525210,5.57342,14.57660,24.0086,True
1,1.2035,0.004385,-0.039214,46,0.001294,0.001039,-0.002358,0.002607,0.000769,0.003727,...,12,0,1.311120,1.183640,0.959407,0.508037,6.48649,12.43240,25.0811,True
2,1.1345,-0.088344,-0.110241,58,-0.000318,-0.000709,-0.002467,0.000483,-0.000484,0.002265,...,1,0,1.079450,1.247460,1.044810,0.634554,0.00000,1.60256,24.8397,True
3,4.0073,-0.275007,-0.439436,49,-0.000068,-0.001143,-0.004597,0.000185,0.009232,0.009872,...,3,0,1.510850,1.186750,1.462440,0.774820,10.85810,21.97900,21.9790,True
4,-4.0166,-0.077677,-0.092900,25,-0.000105,-0.001368,-0.001970,0.000141,-0.003517,-0.005370,...,0,0,1.470630,1.351290,1.151170,0.726595,9.81241,20.77920,20.7792,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,-0.8457,-0.000933,-0.005873,43,0.000334,0.000549,-0.001156,0.001068,0.000214,0.000428,...,2,0,0.720554,0.970544,0.695210,1.051000,0.00000,24.30700,46.6951,True
258,1.5021,-0.002501,-0.007108,42,0.000326,0.000691,-0.000420,0.000963,-0.001287,0.000428,...,1,0,1.355980,0.994171,0.665529,1.095800,0.00000,12.05070,45.4545,True
259,-1.4898,-0.082206,-0.114434,32,0.000000,0.001041,-0.002094,0.000514,-0.001479,-0.002405,...,0,0,0.795768,0.824149,0.768519,0.666445,1.89036,4.63138,21.2665,True
260,0.1638,-0.050526,-0.131182,43,-0.000458,0.000472,-0.002884,0.000084,0.005594,0.011305,...,4,0,0.985386,1.443570,1.424330,0.653009,2.28945,3.18888,29.0270,True


In [15]:

df_mn = df[ df['class0']==1.0 ]
# df_mn = df[ df['class0']!=1.0 ].copy()
# df_mn = df_now
# df_mn = df

X_data = df_mn[X_columns]  
Y_data = df_mn.class0.values.reshape(-1,1)  # 转换成 pd type array
print( X_data.shape, Y_data.shape )

train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data, shuffle=True)
# train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
# train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
# train_ds = TensorDataset(train_x, train_y)
# train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_X = torch.from_numpy(test_x.values).type(torch.float32).to(device)
test_Y = torch.from_numpy(test_y).type(torch.float32).to(device)
# test_ds = TensorDataset(test_x, test_y)
# test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

accuracy(model(test_X), test_Y)

(9420, 18) (9420, 1)


tensor(0.0004, device='cuda:0')

In [16]:
# Y_data
test_Y

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [17]:
""" 模型保存 """
# torch.save( model, '../model/test5-8.pth')
# path = '../model/test5-8'
# torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optim.state_dict(),
#            }, path)

' 模型保存 '

In [18]:
# list1 = []
# for i in range(0,X_app.size(0)):
#     list1.append( random.randint(1,1) )
# len(list1)
# Y_app = torch.tensor( list1 )
# Y_app.to(device)
# Y_app.reshape(-1,1).size(0) 

In [20]:
# list1 = []
# for i in range(0,X_app.size(0)):
#     list1.append( 1.0 )
# Y_app = torch.tensor( list1).type(torch.float32).reshape(-1,1).to(device)
# Y_app.shape


In [21]:
# Y_app = torch.tensor( list1).type(torch.float32)
# X_app.shape, Y_app.shape


In [22]:
# print( '准确率：', accuracy(model(X_app), Y_app) )

In [23]:
# test_y
# Y_app

In [24]:
# X_app
# Y_app.to(device)
# accuracy(model(X_app), Y_app)
# accuracy(model(test_x), test_y)

In [25]:
# Y_app2

In [26]:
# y_pred2 = (Y_app2 > 0.5).type(torch.int32)
# y_pred2

In [27]:
# model = TheModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']

数据准确性验证
过拟合/欠拟合
- 过拟合：对于已知数据过度拟合，对未知数据预测性差
- 欠拟合：对于已知数据拟合不够，对未知数据预测性差
  
tensor合并
- 

In [28]:
# train_x.shape, train_y.shape
# train_cat = torch.cat([train_x, train_y],dim=1)   # 沿着Dimension 1进行合并
# train_cat.shape
# train_all.numpy()
# np.savetxt('../dataset/test20.csv', train_cat.numpy(),fmt='%.8f',delimiter=',') # 直接覆盖
# train_np = np.loadtxt('../dataset/test20.csv', dtype = np.float32, delimiter=',')

In [29]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# HR_dataset = TensorDataset(X, Y)
# test_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

Claude: 根据提供的notebook,Claude2对代码的建议和注释如下:
1. 数据处理部分可以添加更多注释,说明读取的数据字段含义,以及对数据的检查和处理思路。
2. 模型定义部分可以添加注释,说明输入和输出维度的设定依据,网络层数和神经元数量的选择依据等。
3. 训练评估部分的注释可以更详细,说明采用的数据分割方式,训练过程中的超参数,以及评估指标的计算方法。
4. 模型训练结果可以进行更深入的分析,检查是否存在过拟合或欠拟合,并结合业务目标提出后续优化思路。 
5. 可以添加模型效果可视化的代码,如Loss曲线,预测结果示例等,更直观地呈现模型效果。
6. 添加模型保存和加载的代码,便于训练后的模型重用。
7. 可以尝试不同模型结构或训练策略的比较实验。
8. 可以基于代码封装出函数或类,提高代码的复用性和模块化程度。
9. 添加类型注释,可以提高代码可读性和健壮性。
总体而言,该notebook内容完整,但仍有进一步提升注释、可视化、模块化、模型分析等方面的优化空间。